In [1]:
import os
import pdb
import torch
import pandas
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv, GATv2Conv
import torch.nn.functional as F

In [2]:
dataset = Planetoid(root='data/Planetoid', name='CiteSeer', transform=NormalizeFeatures())

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Dataset: CiteSeer():
Number of graphs: 1
Number of features: 3703
Number of classes: 6
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
Number of nodes: 3327
Number of edges: 9104
Average node degree: 2.74
Number of training nodes: 120
Training node label rate: 0.04
Has isolated nodes: True
Has self-loops: False
Is undirected: True


In [3]:
class GATv2(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)
        
        self.conv1 = GATv2Conv(dataset.num_features, hidden_channels, dropout=0.6)
        self.conv2 = GATv2Conv(hidden_channels, dataset.num_classes, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [4]:
model = GATv2(hidden_channels=8, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def test(mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
    acc = int(correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
    return acc

for epoch in range(1, 101):
    loss = train()
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

GATv2(
  (conv1): GATv2Conv(3703, 8, heads=1)
  (conv2): GATv2Conv(8, 6, heads=1)
)
Epoch: 001, Loss: 1.8223, Val: 0.1880, Test: 0.1690
Epoch: 002, Loss: 1.8230, Val: 0.1880, Test: 0.1690
Epoch: 003, Loss: 1.7957, Val: 0.1880, Test: 0.1690
Epoch: 004, Loss: 1.8034, Val: 0.1880, Test: 0.1690
Epoch: 005, Loss: 1.7974, Val: 0.1880, Test: 0.1690
Epoch: 006, Loss: 1.7910, Val: 0.1880, Test: 0.1690
Epoch: 007, Loss: 1.7907, Val: 0.1880, Test: 0.1690
Epoch: 008, Loss: 1.7769, Val: 0.1880, Test: 0.1690
Epoch: 009, Loss: 1.8049, Val: 0.2280, Test: 0.2080
Epoch: 010, Loss: 1.7770, Val: 0.3780, Test: 0.3270
Epoch: 011, Loss: 1.7765, Val: 0.2840, Test: 0.2240
Epoch: 012, Loss: 1.7841, Val: 0.2320, Test: 0.1830
Epoch: 013, Loss: 1.7773, Val: 0.2320, Test: 0.1820
Epoch: 014, Loss: 1.7622, Val: 0.2320, Test: 0.1820
Epoch: 015, Loss: 1.7779, Val: 0.2320, Test: 0.1820
Epoch: 016, Loss: 1.7732, Val: 0.2320, Test: 0.1820
Epoch: 017, Loss: 1.7653, Val: 0.2320, Test: 0.1810
Epoch: 018, Loss: 1.7792, Val: 0

In [5]:
test_acc = test(data.val_mask)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.5760
